In [173]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
import json
# from prophet import Prophet
import matplotlib.pyplot as plt
import seaborn as sns
import random
#import meteoserver as meteo
#from .autonotebook import tqdm as notebook_tqdm
from datetime import datetime
from pycaret.time_series import *

warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

Train Data Prep

In [174]:
df = pd.read_excel('data_train2022-202404.xlsx')
df_train = df[(df['ds'] >= '2023-01-01 00:00:00') & (df['ds'] <= '2023-12-31 23:00:00')].reset_index(drop = True)
df_train = df_train.rename(columns = {'ptf' : 'y'})
df_train['hour'] =  df_train['ds'].dt.hour
df_train.set_index('ds', inplace=True)
df_train['sun_prod'] = df_train['sun_prod'].astype(float)

Test Data Prep

In [175]:
df_test =  df[(df['ds'] >= '2024-01-01 00:00:00') & (df['ds'] <= '2024-03-01 23:00:00')].reset_index(drop = True)
df_test['hour'] = df_test['ds'].dt.hour

In [178]:
fh = 1*24 
fold = 10
s = setup(df_train, target = 'y', fold = fold, fh = fh)

,Description,Value
0,session_id,6670
1,Target,y
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(8760, 16)"
5,Transformed data shape,"(8760, 16)"
6,Transformed train set shape,"(8736, 16)"
7,Transformed test set shape,"(24, 16)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [179]:
# exog_vars = ['y','sun_prod', 'wind_prod', 'baraj_prod', 'waterfall_prod',
#        'dayofmonth', 'dayofweek', 'quarter', 'month', 'year', 'dayofyear',
#        'weekofyear', 'holiday', 'is_holiday_lead_2', 'hour']

# data = df_train[exog_vars]

# data['sun_prod'] = data['sun_prod'].astype('float64')
# data.info()

In [180]:
best_mape_models_top3 = compare_models(sort = 'MAPE', n_select = 3)
pull(best_mape_models_top3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:42:48
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Linear w/ Cond. Deseasonalize & Detrending


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
auto_arima,Auto ARIMA,1.1187,0.8649,408.4110,473.5871,0.1862,0.2096,-0.1376,0.5500
croston,Croston,1.1265,0.9184,411.2163,502.8231,0.2455,0.2078,-0.1993,0.0260


Processing:   0%|          | 0/83 [00:00<?, ?it/s]

In [ ]:
ensemble_ts = blend_models(best_mape_models_top3)
pull(ensemble_ts)

Final Ensemble Models

In [ ]:
final_ensemble = finalize_model(ensemble_ts)
final_ensemble

In [ ]:
exog_vars = ['sun_prod', 'wind_prod', 'baraj_prod', 'waterfall_prod',
       'dayofmonth', 'dayofweek', 'quarter', 'month', 'year', 'dayofyear',
       'weekofyear', 'holiday', 'is_holiday_lead_2', 'hour']

data = df_train[exog_vars]

In [ ]:
exog_exps = []
target = df_train['y']
exog_models = []
for exog_var in exog_vars:
    exog_exp = TSForecastingExperiment()
    exog_exp.setup(
        data=data[exog_var], fh=10)

    # Users can customize how to model future exogenous variables i.e. add
    # more steps and models to potentially get better models at the expense
    # of higher modeling time.
    best = exog_exp.compare_models(
        sort="mase"#, include=["br_cds_dt", "en_cds_dt", "lar_cds_dt", "ridge_cds_dt", "lightgbm_cds_dt", "arima", "theta"]      
    )
    final_exog_model = exog_exp.finalize_model(best)

    exog_exps.append(exog_exp)
    exog_models.append(final_exog_model)

# Step 2: Get future predictions for exog variables ----
future_exog = [
    exog_exp.predict_model(exog_model)
    for exog_exp, exog_model in zip(exog_exps, exog_models)
]
future_exog = pd.concat(future_exog, axis=1)
future_exog.columns = exog_vars

In [ ]:
exp_future = TSForecastingExperiment()
final_slim_model = final_ensemble
future_preds = exp_future.predict_model(final_slim_model, X=future_exog)

In [ ]:
future_preds.index.name = 'ds'
future_preds = future_preds.reset_index()
future_preds

In [ ]:
df_actual = df_test[['ds', 'ptf']]
df_actual = df_actual[df_actual['ds']<'2024-01-02 00:00:00']
# df_actual.set_index('ds', inplace=True)

In [ ]:
total_error = abs(df_actual['ptf'] - future_preds['y_pred']).sum()
total_y = df_actual['ptf'].sum()
wMAPE = total_error / total_y
wMAPE

In [ ]:
exo_test = df_test[['sun_prod', 'wind_prod', 'baraj_prod', 'waterfall_prod', 'dayofmonth',
       'dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'weekofyear',
       'holiday', 'is_holiday_lead_2', 'hour']]
exo_test = exo_test[:10]
exo_test

In [ ]:
exo_test_feature = exo_test [['sun_prod', 'wind_prod', 'baraj_prod', 'waterfall_prod']]
future_exog_feature = future_exog[['sun_prod', 'wind_prod', 'baraj_prod', 'waterfall_prod']]

wMAPE For Exogenous

In [ ]:
errors = abs(exo_test_feature- future_exog_feature)

# Toplam hataları ve toplam y değerlerini hesaplayın
total_errors = errors.sum()
total_y = exo_test_feature.sum()

# wMAPE hesaplaması
wMAPE = total_errors / total_y

print("wMAPE:\n", wMAPE)

Future Exogenous Forecasting

In [ ]:
future_exog = future_exog.reset_index()
exo_test['ds'] = future_exog['index'].rename('ds')

exo_test['ds'] = exo_test['ds'].dt.to_timestamp()

exo_test = exo_test.drop(columns='ds')
